In [1]:
# import json
import numpy as np
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
import json
import os
import uuid
from PIL import Image
import random
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob

In [2]:
def reset(percent=20):
    return random.randrange(100) < percent

In [3]:
# Training and Validate Sets
# degs = [45,90,135,180,225,270,315]
degs = [90,180,270,360,450,540,630]
img_bbs = []
count = 1
phase_count = {
    "train": 1,
    "test": 1
}

labels = os.path.join('training.json')
with open(labels,'r') as json_file:
    datas = json.load(json_file)
    for data in datas:
        read_path = 'tmp_images' + data['image']['pathname']
        print(read_path)
        phase = "train" if (count % 2) == 0 else "test"            
        img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
        print(img_name_new)
        save_path = os.path.join('../data/custom/images',img_name_new+".jpg")
        img = Image.open(read_path)
        objects = data['objects']
        rects = []
        result = ""
        img = np.array(img)
        h,w,c = img.shape
        RBCs_flag = reset(17)
        for obj in objects:
            minr = obj['bounding_box']['minimum']['r']
            minc = obj['bounding_box']['minimum']['c']
            maxr = obj['bounding_box']['maximum']['r']
            maxc = obj['bounding_box']['maximum']['c']
            x1 = minc / w
            y1 = minr / h
            bw = (maxc - minc) / w
            bh = (maxr - minr) / h
            xc = x1 + (bw / 2)
            yc = y1 + (bh / 2)
            category = obj['category']
            if (category == "red blood cell") and (RBCs_flag or phase == "test"):
                result += "\n"+ "0 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            elif category == "ring" or category == "trophozoite" or category == "schizont" or category == "gametocyte":
                result += "\n" + "1 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                rects.append(BoundingBox(x1=minc,y1=minr,x2=maxc,y2=maxr,label=1))
        result = result[1:]
        if len(result) == 0:
            count += 1
            phase_count[phase] += 1
            continue
        Image.fromarray(img).save(save_path)
        f = open(os.path.join('../data/custom/labels',img_name_new+".txt"),'w')
        f.write(result)
        f.close()
        bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        count += 1
        phase_count[phase] += 1
        if phase == "train":
            for deg in degs:
                image_aug, bbs_aug = iaa.Affine(rotate=deg,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img, bounding_boxes=bbs)
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                h,w,c = image_aug.shape
                result = ""
                img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
                for bb in clip_off_bbs.bounding_boxes:
                    minx = int(bb.x1)
                    miny = int(bb.y1)
                    maxx = int(bb.x2)
                    maxy = int(bb.y2)
                    bb_w = (maxx-minx)/w
                    bb_h = (maxy-miny)/h
                    bb_xc = (minx + ((maxx-minx)/2))/w
                    bb_yc = (miny + ((maxy-miny)/2))/h
                    result += "\n" + str(bb.label)+" "+str(bb_xc)+" "+str(bb_yc)+ " "+str(bb_w)+ " "+str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    print("{} {}".format(phase_count[phase],deg))
                    print(img_name_new)
                    result = result[1:]
                    save_path = os.path.join('../data/custom/images',img_name_new+".jpg")
                    Image.fromarray(image_aug).save(save_path)
                    f = open(os.path.join('../data/custom/labels',img_name_new+".txt"),'w')
                    f.write(result)
                    f.close()
                    phase_count[phase] += 1

tmp_images/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png
pv-test-0000000001
tmp_images/images/10be6380-cbbb-4886-8b9e-ff56b1710576.png
pv-train-0000000001
tmp_images/images/6b14c855-8561-417c-97a4-63fa552842fd.png
pv-test-0000000002
tmp_images/images/13099edb-35d9-438f-b093-2cf2ebf9d255.png
pv-train-0000000002
3 90
pv-train-0000000003
4 180
pv-train-0000000004
5 360
pv-train-0000000005
6 450
pv-train-0000000006
7 540
pv-train-0000000007
8 630
pv-train-0000000008
tmp_images/images/2559636b-f01a-4414-93da-210c3b12d153.png
pv-test-0000000003
tmp_images/images/0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png
pv-train-0000000009
10 90
pv-train-0000000010
11 180
pv-train-0000000011
12 270
pv-train-0000000012
13 360
pv-train-0000000013
14 450
pv-train-0000000014
15 540
pv-train-0000000015
16 630
pv-train-0000000016
tmp_images/images/7c66d2fd-68ee-4628-b860-9d7ad25b2a80.png
pv-test-0000000004
tmp_images/images/84c88143-940d-4b62-bbd2-7acc4cadcb6b.png
pv-train-0000000017
tmp_images/images/c6d97845-8a

143 540
pv-train-0000000143
144 630
pv-train-0000000144
tmp_images/images/0f26699f-a776-4424-b092-27da9a1d12e2.png
pv-test-0000000035
tmp_images/images/98efca97-37d2-4459-aca6-375ecc0c4539.png
pv-train-0000000145
146 90
pv-train-0000000146
147 180
pv-train-0000000147
148 360
pv-train-0000000148
149 540
pv-train-0000000149
tmp_images/images/f1de7c7e-e7d3-45ae-a66c-58538bf6cac1.png
pv-test-0000000036
tmp_images/images/9615ff33-b3fc-4d1e-9c05-0b8e3ef936df.png
pv-train-0000000150
tmp_images/images/a2d07312-2f97-40d2-8b09-7f2aca7b16fb.png
pv-test-0000000037
tmp_images/images/b91b1641-ee06-44d8-94cc-24cdb583cc37.png
pv-train-0000000151
152 90
pv-train-0000000152
153 180
pv-train-0000000153
154 270
pv-train-0000000154
155 360
pv-train-0000000155
156 450
pv-train-0000000156
157 540
pv-train-0000000157
158 630
pv-train-0000000158
tmp_images/images/3fa1a4e3-c80d-49da-8f7b-a1abb0424f48.png
pv-test-0000000038
tmp_images/images/c6ab535c-30e5-4b6b-8273-1ec8d78cdd13.png
pv-train-0000000159
tmp_images

tmp_images/images/933e7e07-fc3d-42eb-be0c-6b1623ee60fa.png
pv-train-0000000305
tmp_images/images/694222bb-cddf-469a-9526-f873c6fba58e.png
pv-test-0000000065
tmp_images/images/6bb4892c-27a9-4642-aba1-4b73f38f757c.png
pv-train-0000000306
307 270
pv-train-0000000307
308 360
pv-train-0000000308
309 630
pv-train-0000000309
tmp_images/images/65e3d6a6-012c-4b40-9a42-83078eb85e31.png
pv-test-0000000066
tmp_images/images/9a1cd456-310c-47b6-a1f8-0fbd8e33d1fd.png
pv-train-0000000310
tmp_images/images/ae779806-6d01-4dc6-89df-294c368db6bc.png
pv-test-0000000067
tmp_images/images/e234ce8b-4ec4-4d93-9068-d7854d77ddf4.png
pv-train-0000000311
tmp_images/images/dfc026a9-079f-4f45-8666-e0ccba5bb437.png
pv-test-0000000068
tmp_images/images/a6aa3b72-2970-4c78-b655-ea8dfe6e018c.png
pv-train-0000000312
313 90
pv-train-0000000313
314 180
pv-train-0000000314
315 270
pv-train-0000000315
316 360
pv-train-0000000316
317 450
pv-train-0000000317
318 540
pv-train-0000000318
319 630
pv-train-0000000319
tmp_images/ima

tmp_images/images/313ed8d9-3c00-4415-9de0-0eda07c60ad9.png
pv-test-0000000096
tmp_images/images/5a0628af-4c0a-4ba6-a935-4ec94c9c700e.png
pv-train-0000000456
tmp_images/images/3b55c42f-ad59-422f-ba99-4f4f4ca9cb2e.png
pv-test-0000000097
tmp_images/images/cc94eabb-a3dc-4cfc-9fff-55c76b5b4a71.png
pv-train-0000000457
458 90
pv-train-0000000458
459 180
pv-train-0000000459
460 270
pv-train-0000000460
461 360
pv-train-0000000461
462 450
pv-train-0000000462
463 540
pv-train-0000000463
464 630
pv-train-0000000464
tmp_images/images/8e4bdd0d-1c72-4836-ae3e-05e2e50e4983.png
pv-test-0000000098
tmp_images/images/0901f9d1-be4f-4c19-8e83-dd724dded9f5.png
pv-train-0000000465
466 90
pv-train-0000000466
467 180
pv-train-0000000467
468 270
pv-train-0000000468
469 360
pv-train-0000000469
470 450
pv-train-0000000470
471 540
pv-train-0000000471
472 630
pv-train-0000000472
tmp_images/images/5a043d69-f435-46d4-8691-395c1602a76a.png
pv-test-0000000099
tmp_images/images/68442532-9cd2-4b42-9d4f-ddf8c13bb38c.png
pv

612 450
pv-train-0000000612
613 540
pv-train-0000000613
tmp_images/images/e474a20b-2769-48f1-a520-4f6ef5a0dbf5.png
pv-test-0000000126
tmp_images/images/9e5437c6-b0d3-473b-b7c3-8f5afeb90c7a.png
pv-train-0000000614
615 90
pv-train-0000000615
616 180
pv-train-0000000616
617 270
pv-train-0000000617
618 360
pv-train-0000000618
619 450
pv-train-0000000619
620 540
pv-train-0000000620
tmp_images/images/1315c2a4-0794-4dfa-a709-e2451723d2b6.png
pv-test-0000000127
tmp_images/images/69565501-6e77-4f40-bd80-caa3dda34d8b.png
pv-train-0000000621
tmp_images/images/c7045ab6-230e-4c1e-a8e0-f624d41a00d9.png
pv-test-0000000128
tmp_images/images/a27858ef-6533-413e-bc75-cc54690261a2.png
pv-train-0000000622
623 90
pv-train-0000000623
624 180
pv-train-0000000624
625 270
pv-train-0000000625
626 360
pv-train-0000000626
627 450
pv-train-0000000627
628 540
pv-train-0000000628
629 630
pv-train-0000000629
tmp_images/images/bc377041-e098-45c4-9651-2b1ea90d1d7e.png
pv-test-0000000129
tmp_images/images/d6352a03-ef81-4

...

2490 90
pv-train-0000002490
2491 180
pv-train-0000002491
2492 270
pv-train-0000002492
2493 360
pv-train-0000002493
2494 450
pv-train-0000002494
2495 540
pv-train-0000002495
2496 630
pv-train-0000002496
tmp_images/images/9af1ee02-87fd-42bc-963e-fa9ac3eb45bb.png
pv-test-0000000481
tmp_images/images/7dfbfeaf-e3b1-4159-8dcd-d7f731280ac8.png
pv-train-0000002497
2498 90
pv-train-0000002498
2499 180
pv-train-0000002499
2500 270
pv-train-0000002500
2501 360
pv-train-0000002501
2502 450
pv-train-0000002502
2503 540
pv-train-0000002503
2504 630
pv-train-0000002504
tmp_images/images/9ecf2006-8d49-4ae8-a14b-ed8b3b14ee7c.png
pv-test-0000000482
tmp_images/images/f7a4334a-4332-4de8-b39b-22da907976f2.png
pv-train-0000002505
2506 90
pv-train-0000002506
2507 180
pv-train-0000002507
2508 270
pv-train-0000002508
2509 360
pv-train-0000002509
2510 450
pv-train-0000002510
2511 540
pv-train-0000002511
2512 630
pv-train-0000002512
tmp_images/images/9e1b0217-0926-4fd1-bb9e-a815c05deb59.png
pv-test-0000000483
tm

2670 90
pv-train-0000002670
2671 180
pv-train-0000002671
2672 270
pv-train-0000002672
2673 360
pv-train-0000002673
2674 450
pv-train-0000002674
2675 540
pv-train-0000002675
2676 630
pv-train-0000002676
tmp_images/images/191f87a4-6f2d-49e7-bba0-08e3aa78ddaf.png
pv-test-0000000505
tmp_images/images/14005767-9296-4b48-8a9d-bbe9b2040005.png
pv-train-0000002677
2678 180
pv-train-0000002678
2679 270
pv-train-0000002679
2680 360
pv-train-0000002680
2681 540
pv-train-0000002681
tmp_images/images/e8058bed-c362-4e1c-85dc-62f80f32938f.png
pv-test-0000000506
tmp_images/images/fcdf6632-c03f-49a8-9fb3-42e87c99b9d0.png
pv-train-0000002682
2683 90
pv-train-0000002683
2684 180
pv-train-0000002684
2685 270
pv-train-0000002685
2686 360
pv-train-0000002686
2687 450
pv-train-0000002687
2688 540
pv-train-0000002688
2689 630
pv-train-0000002689
tmp_images/images/e5fd5949-4cd0-4cb6-837a-02c26a5eb87a.png
pv-test-0000000507
tmp_images/images/5f66d20c-fbfe-44ec-a01b-8a09da24807e.png
pv-train-0000002690
2691 90
p

2845 540
pv-train-0000002845
2846 630
pv-train-0000002846
tmp_images/images/fde0fd54-49cc-42e1-90c6-6b612cd062d8.png
pv-test-0000000530
tmp_images/images/664c570b-4822-413c-9724-11b1d91fac44.png
pv-train-0000002847
2848 90
pv-train-0000002848
2849 180
pv-train-0000002849
2850 270
pv-train-0000002850
2851 360
pv-train-0000002851
2852 450
pv-train-0000002852
2853 540
pv-train-0000002853
2854 630
pv-train-0000002854
tmp_images/images/4a49841f-216c-4815-8140-b8762c4f6603.png
pv-test-0000000531
tmp_images/images/f0713bee-2a71-4ced-b953-4041ea99496a.png
pv-train-0000002855
2856 90
pv-train-0000002856
2857 180
pv-train-0000002857
2858 270
pv-train-0000002858
2859 360
pv-train-0000002859
2860 450
pv-train-0000002860
2861 540
pv-train-0000002861
2862 630
pv-train-0000002862
tmp_images/images/2bd15b6e-fe2d-49f3-b394-940d024d1f91.png
pv-test-0000000532
tmp_images/images/9edba799-e169-46cd-b100-c9f8fc7a8c7d.png
pv-train-0000002863
tmp_images/images/089c9744-f906-41d8-908d-1e37919a8658.png
pv-test-

tmp_images/images/c890ee00-daa5-4b20-9cd5-930748ecc4e0.png
pv-train-0000003019
3020 90
pv-train-0000003020
3021 180
pv-train-0000003021
3022 270
pv-train-0000003022
3023 360
pv-train-0000003023
3024 450
pv-train-0000003024
3025 540
pv-train-0000003025
3026 630
pv-train-0000003026
tmp_images/images/58594235-0d32-4a4e-91cd-4c800eaaaa73.png
pv-test-0000000556
tmp_images/images/5f2b32bd-fdca-4d6f-b13a-ee9f83626c8d.png
pv-train-0000003027
3028 90
pv-train-0000003028
3029 180
pv-train-0000003029
3030 270
pv-train-0000003030
3031 360
pv-train-0000003031
3032 450
pv-train-0000003032
3033 540
pv-train-0000003033
3034 630
pv-train-0000003034
tmp_images/images/7624ac36-1cf9-42fb-844d-376078309553.png
pv-test-0000000557
tmp_images/images/a51d6c18-f9e8-4714-9dbb-38e1577445f2.png
pv-train-0000003035
3036 90
pv-train-0000003036
3037 180
pv-train-0000003037
3038 270
pv-train-0000003038
3039 360
pv-train-0000003039
3040 450
pv-train-0000003040
3041 540
pv-train-0000003041
3042 630
pv-train-0000003042
t

3197 450
pv-train-0000003197
3198 540
pv-train-0000003198
3199 630
pv-train-0000003199
tmp_images/images/2165fe38-b474-467d-be11-d1d728085efd.png
pv-test-0000000580
tmp_images/images/25ddc1ca-d368-4fff-b1ad-1dbf1e7c97aa.png
pv-train-0000003200
tmp_images/images/51e86aa9-caa0-4297-94e8-34a57d7f7e36.png
pv-test-0000000581
tmp_images/images/637957ef-df75-4a7b-86d5-9f6e9c042ca2.png
pv-train-0000003201
tmp_images/images/a0e2ef53-d39b-40cf-b8b6-4b148ac339ec.png
pv-test-0000000582
tmp_images/images/dd6f047e-af73-43df-9c46-354534fc20e0.png
pv-train-0000003202
tmp_images/images/fe691faa-3bc7-475e-b558-aa8fe7b795bc.png
pv-test-0000000583
tmp_images/images/07a80340-f65e-4f58-9317-94bb6ebd5a28.png
pv-train-0000003203
3204 90
pv-train-0000003204
3205 180
pv-train-0000003205
3206 270
pv-train-0000003206
3207 360
pv-train-0000003207
3208 450
pv-train-0000003208
3209 540
pv-train-0000003209
3210 630
pv-train-0000003210
tmp_images/images/3ff98212-7c2a-46cd-afb7-34a2479bed07.png
pv-test-0000000584
tmp_i

In [4]:
print(phase_count["train"])
print(phase_count["test"])

3365
605


In [5]:
# Training and Validate Sets
# degs = [45,90,135,180,225,270,315]
img_bbs = []
# phase_count["train"] = 3484
labels = os.path.join('test.json')
with open(labels,'r') as json_file:
    datas = json.load(json_file)
    for data in datas:
        read_path = 'tmp_images' + data['image']['pathname']
        print(read_path)
        phase = "train"
        img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
        print(img_name_new)
        save_path = os.path.join('../data/custom/images',img_name_new+".jpg")
        img = Image.open(read_path)
        objects = data['objects']
        rects = []
        result = ""
        img = np.array(img)
        h,w,c = img.shape
        for obj in objects:
            minr = obj['bounding_box']['minimum']['r']
            minc = obj['bounding_box']['minimum']['c']
            maxr = obj['bounding_box']['maximum']['r']
            maxc = obj['bounding_box']['maximum']['c']
            x1 = minc / w
            y1 = minr / h
            bw = (maxc - minc) / w
            bh = (maxr - minr) / h
            xc = x1 + (bw / 2)
            yc = y1 + (bh / 2)
            category = obj['category']
            if (category == "red blood cell"):
                result += "\n"+ "0 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            elif category == "ring" or category == "trophozoite" or category == "schizont" or category == "gametocyte":
                result += "\n" + "1 " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                rects.append(BoundingBox(x1=minc,y1=minr,x2=maxc,y2=maxr,label=1))
        result = result[1:]
        Image.fromarray(img).save(save_path)
        f = open(os.path.join('../data/custom/labels',img_name_new+".txt"),'w')
        f.write(result)
        f.close()
        bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        count += 1
        phase_count[phase] += 1
        if phase == "train":
            for deg in degs:
                image_aug, bbs_aug = iaa.Affine(rotate=deg,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img, bounding_boxes=bbs)
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                h,w,c = image_aug.shape
                result = ""
                img_name_new = "pv-" + phase + "-{:010d}".format(phase_count[phase])
                
                for bb in clip_off_bbs.bounding_boxes:
                    minx = int(bb.x1)
                    miny = int(bb.y1)
                    maxx = int(bb.x2)
                    maxy = int(bb.y2)
                    bb_w = (maxx-minx)/w
                    bb_h = (maxy-miny)/h
                    bb_xc = (minx + ((maxx-minx)/2))/w
                    bb_yc = (miny + ((maxy-miny)/2))/h
                    result += "\n" + str(bb.label)+" "+str(bb_xc)+" "+str(bb_yc)+ " "+str(bb_w)+ " "+str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    print("{} {}".format(phase_count[phase],deg))
                    print(img_name_new)
                    result = result[1:]
                    save_path = os.path.join('../data/custom/images',img_name_new+".jpg")
                    Image.fromarray(image_aug).save(save_path)
                    f = open(os.path.join('../data/custom/labels',img_name_new+".txt"),'w')
                    f.write(result)
                    f.close()
                    phase_count[phase] += 1

tmp_images/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg
pv-train-0000003365
3366 90
pv-train-0000003366
3367 180
pv-train-0000003367
3368 270
pv-train-0000003368
3369 360
pv-train-0000003369
3370 450
pv-train-0000003370
3371 540
pv-train-0000003371
3372 630
pv-train-0000003372
tmp_images/images/64985a1e-16bb-4016-a01c-c21a5b86e572.jpg
pv-train-0000003373
3374 90
pv-train-0000003374
3375 180
pv-train-0000003375
3376 270
pv-train-0000003376
3377 360
pv-train-0000003377
3378 450
pv-train-0000003378
3379 540
pv-train-0000003379
3380 630
pv-train-0000003380
tmp_images/images/c70894fa-50e8-4eed-a5a8-159a20153a49.jpg
pv-train-0000003381
3382 90
pv-train-0000003382
3383 180
pv-train-0000003383
3384 270
pv-train-0000003384
3385 360
pv-train-0000003385
3386 450
pv-train-0000003386
3387 540
pv-train-0000003387
3388 630
pv-train-0000003388
tmp_images/images/abb72ce2-4193-4dd3-a34c-61c2ee22ccff.jpg
pv-train-0000003389
3390 90
pv-train-0000003390
3391 180
pv-train-0000003391
3392 270
pv-train-000

3596 180
pv-train-0000003596
3597 270
pv-train-0000003597
3598 360
pv-train-0000003598
3599 450
pv-train-0000003599
3600 540
pv-train-0000003600
3601 630
pv-train-0000003601
tmp_images/images/8111cff9-4c04-448e-990b-9d5010f7be56.jpg
pv-train-0000003602
3603 90
pv-train-0000003603
3604 180
pv-train-0000003604
3605 270
pv-train-0000003605
3606 360
pv-train-0000003606
3607 450
pv-train-0000003607
3608 540
pv-train-0000003608
tmp_images/images/e6a7743e-4040-40fa-b097-ca6874c469b7.jpg
pv-train-0000003609
3610 90
pv-train-0000003610
3611 180
pv-train-0000003611
3612 270
pv-train-0000003612
3613 360
pv-train-0000003613
3614 450
pv-train-0000003614
3615 540
pv-train-0000003615
3616 630
pv-train-0000003616
tmp_images/images/e02301ac-68f7-4daf-b2ba-254b917f1cca.jpg
pv-train-0000003617
3618 90
pv-train-0000003618
3619 180
pv-train-0000003619
3620 270
pv-train-0000003620
3621 360
pv-train-0000003621
3622 450
pv-train-0000003622
3623 540
pv-train-0000003623
3624 630
pv-train-0000003624
tmp_images/i

3828 90
pv-train-0000003828
3829 180
pv-train-0000003829
3830 270
pv-train-0000003830
3831 360
pv-train-0000003831
3832 450
pv-train-0000003832
3833 540
pv-train-0000003833
3834 630
pv-train-0000003834
tmp_images/images/294bc238-efdb-4f88-b83b-f41fe5750106.jpg
pv-train-0000003835
3836 90
pv-train-0000003836
3837 180
pv-train-0000003837
3838 270
pv-train-0000003838
3839 360
pv-train-0000003839
3840 450
pv-train-0000003840
3841 540
pv-train-0000003841
3842 630
pv-train-0000003842
tmp_images/images/34950f4e-fe44-49ec-9d49-14bd4a8ce7c0.jpg
pv-train-0000003843
3844 90
pv-train-0000003844
3845 180
pv-train-0000003845
3846 270
pv-train-0000003846
3847 360
pv-train-0000003847
3848 450
pv-train-0000003848
3849 540
pv-train-0000003849
3850 630
pv-train-0000003850
tmp_images/images/224c80b8-6c55-48cf-b719-96a47d223dc3.jpg
pv-train-0000003851
3852 90
pv-train-0000003852
3853 180
pv-train-0000003853
3854 270
pv-train-0000003854
3855 360
pv-train-0000003855
3856 450
pv-train-0000003856
3857 540
pv-t

4060 270
pv-train-0000004060
4061 360
pv-train-0000004061
4062 450
pv-train-0000004062
4063 540
pv-train-0000004063
4064 630
pv-train-0000004064
tmp_images/images/6040cf0d-9dd6-4798-9406-1afbbcd24d4c.jpg
pv-train-0000004065
4066 90
pv-train-0000004066
4067 180
pv-train-0000004067
4068 270
pv-train-0000004068
4069 360
pv-train-0000004069
4070 450
pv-train-0000004070
4071 630
pv-train-0000004071
tmp_images/images/408745a6-79fb-4804-9c24-e2d6673f8f2e.jpg
pv-train-0000004072
4073 90
pv-train-0000004073
4074 180
pv-train-0000004074
4075 270
pv-train-0000004075
4076 360
pv-train-0000004076
4077 450
pv-train-0000004077
4078 540
pv-train-0000004078
4079 630
pv-train-0000004079
tmp_images/images/010961af-b38c-49de-aca0-e3732d73d414.jpg
pv-train-0000004080
4081 90
pv-train-0000004081
4082 180
pv-train-0000004082
4083 270
pv-train-0000004083
4084 360
pv-train-0000004084
4085 450
pv-train-0000004085
4086 540
pv-train-0000004086
4087 630
pv-train-0000004087
tmp_images/images/47112c6b-aaf6-488d-8f3b

In [6]:
print(phase_count["train"])
print(phase_count["test"])

4264
605


In [7]:
# get images from pv_test (Only test)
labels = os.path.join('pv_test.json')

# phase_count["test"] = 605

phase = "test"
# degs = [45,90,135,180,225,270,315]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pv-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1

1329 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/1.jpg
pv-test-0000000605
1330 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/10.jpg
pv-test-0000000606
1331 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/13.jpg
pv-test-0000000607
1332 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/17.jpg
pv-test-0000000608
1333 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/25.jpg
pv-test-0000000609
1334 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/26.jpg
pv-test-0000000610
1335 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/30.jpg
pv-test-0000000611
1336 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/32.jpg
pv-test-0000000612
1337 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv_test/33.jpg
pv-test-0000000613
1338 https://raw.githubuserco

In [8]:
print(phase_count["train"])
print(phase_count["test"])

4264
633


In [9]:
labels = os.path.join('pv_ext.json')
phase = "train"
# degs = [45,90,135,180,225,270,315]
# phase_count = {
#     "train": 4385,
#     "test": 633
# }

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pv-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(phase_count[phase],url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            phase_count[phase] += 1
            for deg in degs:
                name = "pv-" + phase + "-{:010d}".format(phase_count[phase])
                image_aug, bbs_aug = iaa.Affine(rotate=deg,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img, bounding_boxes=bbs)
                aug_h, aug_w, aug_c = image_aug.shape
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                bboxs = ""
                for bb in clip_off_bbs.bounding_boxes:
                    x1 = bb.x1
                    y1 = bb.y1
                    x2 = bb.x2
                    y2 = bb.y2
                    bb_w = (x2-x1)/aug_w
                    bb_h = (y2-y1)/aug_h
                    bb_xc = (x1/aug_w) + (bb_w/2)
                    bb_yc = (y1/aug_h) + (bb_h/2)
                    bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    bboxs = bboxs[1:]
                    im_aug = Image.fromarray(image_aug)
                    im_aug.save(os.path.join('../data/custom/images',name+".jpg"))
                    f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
                    f.write(bboxs)
                    f.close()
                    print("{} {}".format(phase_count[phase],deg))
                    phase_count[phase] += 1

4264 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000001.jpg
pv-train-0000004264
4265 90
4266 180
4267 270
4268 360
4269 450
4270 540
4271 630
4272 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000002.jpg
pv-train-0000004272
4273 90
4274 180
4275 270
4276 360
4277 450
4278 540
4279 630
4280 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000003.jpg
pv-train-0000004280
4281 90
4282 180
4283 270
4284 360
4285 450
4286 540
4287 630
4288 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000004.jpg
pv-train-0000004288
4289 90
4290 180
4291 270
4292 360
4293 450
4294 540
4295 630
4296 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000006.jpg
pv-train-0000004296
4297 90
4298 180
4299 270
4300 360
4301 450
4302 540
4303 630
4304 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000007.jpg
pv-train-0000004304
4305 90
4306 180
4

4628 90
4629 540
4630 630
4631 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000051.jpg
pv-train-0000004631
4632 90
4633 180
4634 270
4635 360
4636 450
4637 540
4638 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000052.jpg
pv-train-0000004638
4639 90
4640 180
4641 450
4642 540
4643 630
4644 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000053.jpg
pv-train-0000004644
4645 90
4646 180
4647 270
4648 360
4649 450
4650 540
4651 630
4652 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000054.jpg
pv-train-0000004652
4653 90
4654 180
4655 270
4656 360
4657 450
4658 540
4659 630
4660 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000055.jpg
pv-train-0000004660
4661 90
4662 180
4663 270
4664 360
4665 450
4666 540
4667 630
4668 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000056.jpg
pv-train-0000004668
4669 90
4670 180
46

4949 630
4950 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000103.jpg
pv-train-0000004950
4951 270
4952 540
4953 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000104.jpg
pv-train-0000004953
4954 90
4955 180
4956 360
4957 540
4958 630
4959 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000105.jpg
pv-train-0000004959
4960 90
4961 180
4962 360
4963 540
4964 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000106.jpg
pv-train-0000004964
4965 90
4966 180
4967 270
4968 360
4969 450
4970 540
4971 630
4972 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000107.jpg
pv-train-0000004972
4973 90
4974 180
4975 270
4976 360
4977 540
4978 630
4979 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000108.jpg
pv-train-0000004979
4980 90
4981 180
4982 270
4983 450
4984 630
4985 https://raw.githubusercontent.com/RTANC/malaria_dataset/ma

5261 90
5262 180
5263 270
5264 360
5265 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000156.jpg
pv-train-0000005265
5266 90
5267 180
5268 270
5269 360
5270 450
5271 540
5272 630
5273 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000157.jpg
pv-train-0000005273
5274 90
5275 180
5276 270
5277 360
5278 450
5279 540
5280 630
5281 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000158.jpg
pv-train-0000005281
5282 540
5283 630
5284 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000159.jpg
pv-train-0000005284
5285 90
5286 180
5287 270
5288 360
5289 450
5290 540
5291 630
5292 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000160.jpg
pv-train-0000005292
5293 90
5294 180
5295 270
5296 360
5297 450
5298 540
5299 630
5300 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000161.jpg
pv-train-0000005300
5301 90
5302 180
5303 270
5

5574 90
5575 270
5576 450
5577 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000209.jpg
pv-train-0000005577
5578 90
5579 180
5580 270
5581 360
5582 450
5583 630
5584 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000210.jpg
pv-train-0000005584
5585 90
5586 180
5587 270
5588 360
5589 450
5590 540
5591 630
5592 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000211.jpg
pv-train-0000005592
5593 90
5594 180
5595 270
5596 360
5597 450
5598 540
5599 630
5600 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000212.jpg
pv-train-0000005600
5601 90
5602 180
5603 360
5604 450
5605 540
5606 630
5607 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000213.jpg
pv-train-0000005607
5608 90
5609 180
5610 270
5611 360
5612 450
5613 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000214.jpg
pv-train-0000005613
5614 90
5615 180
5616 270
56

...

7831 90
7832 270
7833 360
7834 450
7835 630
7836 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000587.jpg
pv-train-0000007836
7837 270
7838 450
7839 540
7840 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000588.jpg
pv-train-0000007840
7841 90
7842 180
7843 270
7844 360
7845 450
7846 540
7847 630
7848 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000589.jpg
pv-train-0000007848
7849 90
7850 270
7851 360
7852 450
7853 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000590.jpg
pv-train-0000007853
7854 90
7855 270
7856 360
7857 630
7858 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000591.jpg
pv-train-0000007858
7859 90
7860 180
7861 270
7862 360
7863 540
7864 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000592.jpg
pv-train-0000007864
7865 90
7866 180
7867 270
7868 360
7869 450
7870 540
7871 630
7872 https://raw.gi

8162 90
8163 180
8164 270
8165 360
8166 450
8167 540
8168 630
8169 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000639.jpg
pv-train-0000008169
8170 90
8171 180
8172 270
8173 360
8174 450
8175 540
8176 630
8177 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000640.jpg
pv-train-0000008177
8178 180
8179 360
8180 540
8181 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000641.jpg
pv-train-0000008181
8182 90
8183 180
8184 270
8185 360
8186 450
8187 540
8188 630
8189 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000642.jpg
pv-train-0000008189
8190 90
8191 180
8192 360
8193 450
8194 540
8195 630
8196 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000643.jpg
pv-train-0000008196
8197 90
8198 180
8199 270
8200 360
8201 450
8202 540
8203 630
8204 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000644.jpg
pv-train-0000008204


8516 90
8517 180
8518 360
8519 540
8520 630
8521 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000689.jpg
pv-train-0000008521
8522 90
8523 180
8524 270
8525 450
8526 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000690.jpg
pv-train-0000008526
8527 90
8528 180
8529 270
8530 450
8531 540
8532 630
8533 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000691.jpg
pv-train-0000008533
8534 630
8535 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000692.jpg
pv-train-0000008535
8536 90
8537 180
8538 360
8539 450
8540 540
8541 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000693.jpg
pv-train-0000008541
8542 90
8543 180
8544 270
8545 450
8546 540
8547 630
8548 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000694.jpg
pv-train-0000008548
8549 90
8550 180
8551 270
8552 360
8553 450
8554 540
8555 630
8556 https://raw.githubuserc

8849 450
8850 540
8851 630
8852 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000740.jpg
pv-train-0000008852
8853 90
8854 180
8855 270
8856 450
8857 540
8858 630
8859 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000741.jpg
pv-train-0000008859
8860 180
8861 270
8862 360
8863 450
8864 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000742.jpg
pv-train-0000008864
8865 90
8866 180
8867 270
8868 360
8869 450
8870 630
8871 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000743.jpg
pv-train-0000008871
8872 90
8873 180
8874 270
8875 360
8876 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000744.jpg
pv-train-0000008876
8877 90
8878 180
8879 270
8880 360
8881 450
8882 540
8883 630
8884 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000745.jpg
pv-train-0000008884
8885 https://raw.githubusercontent.com/RTANC/malaria_dataset/m

9182 630
9183 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000791.jpg
pv-train-0000009183
9184 90
9185 180
9186 270
9187 360
9188 450
9189 540
9190 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000792.jpg
pv-train-0000009190
9191 180
9192 540
9193 630
9194 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000793.jpg
pv-train-0000009194
9195 90
9196 180
9197 270
9198 360
9199 450
9200 540
9201 630
9202 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000794.jpg
pv-train-0000009202
9203 90
9204 270
9205 360
9206 450
9207 540
9208 630
9209 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000795.jpg
pv-train-0000009209
9210 90
9211 180
9212 270
9213 360
9214 450
9215 540
9216 630
9217 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Vivax/pv-0000000796.jpg
pv-train-0000009217
9218 90
9219 270
9220 450
9221 540
9222 https://raw.githubuser

In [10]:
print(phase_count["train"])
print(phase_count["test"])

9413
633


In [11]:
# # summary test set
# files = glob.glob("images/pv-test-*.jpg")
# count = 0
# count_class = [0,0]

# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     lines = list(f)
#     f.close()
#     print(img)
#     for line in lines:
# #         print(line)
#         class_id, xc, yc, wb, hb = line.split()
#         count_class[int(class_id)] += 1
#     count += 1
# print(count)
# print(count_class)

In [12]:
# print(count)
# print(count_class)

In [13]:
# # summary train set
# files = glob.glob("images/pv-train-*.jpg")
# count = 0
# count_class = [0,0]

# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     lines = list(f)
#     f.close()
#     print(img)
#     for line in lines:
# #         print(line)
#         class_id, xc, yc, wb, hb = line.split()
#         count_class[int(class_id)] += 1
#     count += 1
# print(count)
# print(count_class)

In [14]:
# print(count)
# print(count_class)

In [15]:
# clear empty box
# files = glob.glob("images/pv-train-*.jpg")
# count = 0
# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     txt = list(f)
#     f.close()
#     if len(txt) == 0:
#         print(img)
#         os.remove("labels/"+name+".txt")
#         os.remove(file)
#         count += 1
# print(count)

In [4]:
# img = imread('tmp_images/images/2559636b-f01a-4414-93da-210c3b12d153.png')
# image_aug = iaa.Affine(rotate=90,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img)
# imsave('aug02.png',image_aug)